# Combine national and NERC data
Take the various file (EIA facilities, EIA state-level totals, EPA emissions, etc) and combine them to determine total generation, CO₂ emissions, etc nationally and at NERC regions.

## Instructions
Make sure the `file_date` parameter below is set to whatever value you would like appended to file names.

In **Calculate the extra gen/fuel consumption at the state levels** add years after 2017 in the `for` loop if needed.

In **Fraction of estimated gen/fuels in each NERC region** update the years that plant ids for annual plants are fetched from EIA-923 files.

The entire notebook can be run at once using *Run All Cells*

In [2]:
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json

idx = pd.IndexSlice
cwd = os.getcwd()
data_path = join(cwd, '..', 'Data storage')

### Date string for filenames
This will be inserted into all filenames (reading and writing)

In [58]:
file_date = '2018-03-06'
export_quarter = '2017 Q4'

In [4]:
%load_ext watermark
%watermark -iv -v

pandas      0.22.0
numpy       1.14.2
json        2.0.9
CPython 3.6.4
IPython 6.2.1


In [5]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [6]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [121]:
%aimport Data.make_data
from Data.make_data import states_in_nerc
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data, add_quarter
%aimport Analysis.index
from Analysis.index import g2lb, change_since_2005, generation_index, reduce_emission_factors
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport util.utils
from util.utils import rename_cols, add_facility_location

## Load data

In [8]:
cwd = os.getcwd()
path = join(data_path, 'Derived data',
            'Facility gen fuels and CO2 {}.csv'.format(file_date))
eia_fac = pd.read_csv(path)

In [9]:
rename_cols(eia_fac)

In [10]:
path = join(data_path, 'Derived data',
            'Monthly EPA emissions {}.csv'.format(file_date))
epa = pd.read_csv(path)

## Adjusted facility emissions and generation
I have both the state and custom fuel categories here, but am only using the state fuel categories for now.

In [11]:
fuel_cat_folder = join(data_path, 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [12]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [13]:
gen_fuels_state.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,COW,2001,1,3,8275496.0,852306.0,8275496.0,31.006900,-88.010300,1,772103776.8,772103776.8,772103776.8,772103776.8
1,COW,2001,1,7,587162.0,45991.0,587162.0,34.012800,-85.970800,1,54782214.6,54782214.6,54782214.6,54782214.6
2,COW,2001,1,8,6910786.0,706857.0,6910786.0,33.644344,-87.196486,1,644776333.8,644776333.8,644776333.8,644776333.8
3,COW,2001,1,10,3108892.0,320454.0,3108892.0,32.601700,-87.781100,1,290059623.6,290059623.6,290059623.6,290059623.6
4,COW,2001,1,26,9745127.0,1005460.0,9745127.0,33.244211,-86.458056,1,909220349.1,909220349.1,909220349.1,909220349.1


## Extra gen/fuels from non-reporting

In [14]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [15]:
cwd = os.getcwd()
path = join(data_path, 'Derived data',
            'EIA country-wide gen fuel CO2 {}.csv'.format(file_date))
eia_total = pd.read_csv(path)

path = join(data_path,
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

### Calculate CO₂, generation, and fuel consumption that is not captured by facilities

In [16]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, ef)

In [17]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16916390.0       1011665.061         13155576.0
          2              14970346.0        911345.865         11576491.0
          3              18320543.0       1153327.913         14764162.0
          4              13530117.0        776339.623         10374008.0
          5              14016024.0        801267.923         10922463.0

In [116]:
extra_co2.head()

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
COW  2001 1           1.611286e+09        1.253069e+09
          2           1.425925e+09        1.102661e+09
          3           1.745032e+09        1.406286e+09
          4           1.288744e+09        9.881243e+08
          5           1.335026e+09        1.040365e+09

## Total CO₂ (national)
Combine adjusted CO₂ at facilities and CO₂ from fuel consumption that is not captured by facility data.

In [18]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [19]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)']
                           .groupby(['year', 'month']).sum())
national_co2.name = 'final co2 (kg)'

In [115]:
national_co2.head()

year  month
2001  1        2.145929e+11
      2        1.799442e+11
      3        1.887242e+11
      4        1.744977e+11
      5        1.888085e+11
Name: final co2 (kg), dtype: float64

## National Index and gen by fuels

### Total national generation (all fuels)
Add generation/fuel consumption reported by facilities (with state-level fuel codes) and extra generation/fuel consumption (which also uses state-level fuel codes).

In [20]:
gen_fuels_state.groupby(['type', 'year', 'month']).sum().head()

plant id  total fuel (mmbtu)  generation (mwh)  \
type year month                                                   
COW  2001 1       8550284        1.848875e+09      1.762754e+08   
          2       8550284        1.568672e+09      1.488241e+08   
          3       8550284        1.626246e+09      1.541157e+08   
          4       8550284        1.471568e+09      1.398943e+08   
          5       8498259        1.595267e+09      1.507916e+08   

                 elec fuel (mmbtu)           lat         lon  quarter  \
type year month                                                         
COW  2001 1           1.810304e+09  25142.050268 -53175.8593      645   
          2           1.536017e+09  25142.050268 -53175.8593      645   
          3           1.591151e+09  25142.050268 -53175.8593      645   
          4           1.439361e+09  25142.050268 -53175.8593     1290   
          5           1.563698e+09  25147.539168 -53176.2288     1290   

                 all fuel fossil co2 (kg)  elec fuel fossil co2 (kg)  \
type year month                                                        
COW  2001 1                  1.755757e+11               1.719629e+11   
          2                  1.490065e+11               1.459476e+11   
          3                  1.544286e+11               1.511424e+11   
          4                  1.397181e+11               1.367018e+11   
          5                  1.515424e+11               1.485856e+11   

                 all fuel total co2 (kg)  elec fuel total co2 (kg)  
type year month                                                     
COW  2001 1                 1.755757e+11              1.719629e+11  
          2                 1.490065e+11              1.459476e+11  
          3                 1.544286e+11              1.511424e+11  
          4                 1.397181e+11              1.367018e+11  
          5                 1.515424e+11              1.485856e+11

In [102]:
extra_gen_fuel.columns

Index(['total fuel (mmbtu)', 'generation (mwh)', 'elec fuel (mmbtu)'], dtype='object')

In [126]:
gen_fuel_cols = ['total fuel (mmbtu)', 'generation (mwh)', 'elec fuel (mmbtu)']

national_gen = (gen_fuels_state
                .groupby(['type', 'year', 'month'])[gen_fuel_cols].sum()
                .add(extra_gen_fuel[gen_fuel_cols], fill_value=0))

In [127]:
national_gen.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1            1.865791e+09       177287111.0       1.823460e+09
          2            1.583642e+09       149735483.0       1.547593e+09
          3            1.644567e+09       155269010.0       1.605915e+09
          4            1.485098e+09       140670652.0       1.449735e+09
          5            1.609283e+09       151592915.0       1.574620e+09

In [128]:
national_gen.reset_index().type.unique()

array(['COW', 'DPV', 'GEO', 'HPS', 'HYC', 'NG', 'NUC', 'OOG', 'OTH', 'PC',
       'PEL', 'SUN', 'WAS', 'WND', 'WWW'], dtype=object)

#### Regroup generation from state codes to my custom fuel codes

In [129]:
national_gen_cat = group_fuel_cats(national_gen.reset_index(), custom_fuel_cat,
                               'type', 'fuel category').set_index(['fuel category', 'year', 'month'])

In [130]:
national_gen_cat.head()

total fuel (mmbtu)  generation (mwh)  \
fuel category year month                                         
Coal          2001 1            1.865791e+09       177287111.0   
                   2            1.583642e+09       149735483.0   
                   3            1.644567e+09       155269010.0   
                   4            1.485098e+09       140670652.0   
                   5            1.609283e+09       151592915.0   

                          elec fuel (mmbtu)  
fuel category year month                     
Coal          2001 1           1.823460e+09  
                   2           1.547593e+09  
                   3           1.605915e+09  
                   4           1.449735e+09  
                   5           1.574620e+09

In [131]:
total_gen = national_gen_cat.groupby(['year', 'month']).sum()

### National Index

In [111]:
national_index = total_gen.copy()
national_index['final co2 (kg)'] = national_co2
national_index['index (g/kwh)'] = (national_index['final co2 (kg)']
                                   / national_index['generation (mwh)'])
national_index.reset_index(inplace=True)
add_quarter(national_index)
g2lb(national_index)
change_since_2005(national_index)

In [112]:
national_index.head()

,year,month,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),final co2 (kg),index (g/kwh),datetime,quarter,index (lb/mwh),change since 2005
0,2001,1,2.548580e+09,3.324839e+08,2.413033e+09,2.145929e+11,645.423413,2001-01-01,1,1422.900456,0.077267
1,2001,2,2.134672e+09,2.829402e+08,2.016862e+09,1.799442e+11,635.979511,2001-02-01,1,1402.080431,0.061504
2,2001,3,2.271867e+09,3.007065e+08,2.145616e+09,1.887242e+11,627.602684,2001-03-01,1,1383.612877,0.047523
3,2001,4,2.119103e+09,2.780781e+08,2.003578e+09,1.744977e+11,627.513263,2001-04-01,2,1383.415739,0.047373
4,2001,5,2.289648e+09,3.004857e+08,2.174221e+09,1.888085e+11,628.344387,2001-05-01,2,1385.248036,0.048761


In [70]:
national_index.head()

,year,month,generation (mwh),final co2 (kg),index (g/kwh),datetime,quarter,index (lb/mwh),change since 2005
0,2001,1,3.324839e+08,2.145929e+11,645.423413,2001-01-01,1,1422.900456,0.077267
1,2001,2,2.829402e+08,1.799442e+11,635.979511,2001-02-01,1,1402.080431,0.061504
2,2001,3,3.007065e+08,1.887242e+11,627.602684,2001-03-01,1,1383.612877,0.047523
3,2001,4,2.780781e+08,1.744977e+11,627.513263,2001-04-01,2,1383.415739,0.047373
4,2001,5,3.004857e+08,1.888085e+11,628.344387,2001-05-01,2,1385.248036,0.048761


In [ ]:
path = join(data_path, 'National data',
            'National index {}.csv'.format(file_date))
national_index.to_csv(path, index=False)

#### files for website

In [72]:
monthly_index = national_index.copy()


quarterly_index = monthly_index.groupby(['year', 'quarter'])['generation (mwh)', 'final co2 (kg)'].sum()
quarterly_index.reset_index(inplace=True)
quarterly_index['index (g/kwh)'] = quarterly_index.loc[:, 'final co2 (kg)'] / quarterly_index.loc[:, 'generation (mwh)']
quarterly_index['year_quarter'] = quarterly_index['year'].astype(str) + ' Q' + quarterly_index['quarter'].astype(str)
change_since_2005(quarterly_index)
g2lb(quarterly_index)

annual_index = quarterly_index.groupby('year')['generation (mwh)', 'final co2 (kg)'].sum()
annual_index.reset_index(inplace=True)
annual_index['index (g/kwh)'] = annual_index.loc[:, 'final co2 (kg)'] / annual_index.loc[:, 'generation (mwh)']

change_since_2005(annual_index)
g2lb(annual_index)

In [73]:
quarterly_index.head()

,year,quarter,generation (mwh),final co2 (kg),index (g/kwh),year_quarter,change since 2005,index (lb/mwh)
0,2001,1,9.161307e+08,5.832613e+11,636.657339,2001 Q1,0.062636,1403.574769
1,2001,2,9.062573e+08,5.680978e+11,626.861502,2001 Q2,0.046286,1381.978866
2,2001,3,1.035004e+09,6.617222e+11,639.342879,2001 Q3,0.067118,1409.495312
3,2001,4,8.791288e+08,5.409828e+11,615.362401,2001 Q4,0.027093,1356.627950
4,2002,1,9.043011e+08,5.475895e+11,605.538917,2002 Q1,0.010696,1334.971096


In [74]:
annual_index.tail()

,year,generation (mwh),final co2 (kg),index (g/kwh),change since 2005,index (lb/mwh)
12,2013,4.065839e+09,2.095628e+12,515.423252,-0.139714,1136.302101
13,2014,4.104659e+09,2.092147e+12,509.700555,-0.149266,1123.685843
14,2015,4.091520e+09,1.967419e+12,480.852898,-0.197415,1060.088300
15,2016,4.095209e+09,1.854639e+12,452.880147,-0.244104,998.419572
16,2017,4.034598e+09,1.769787e+12,438.652664,-0.267851,967.053663


In [75]:
path = join(data_path, 'Website results',
            'Monthly index {}.csv'.format(export_quarter))
monthly_index.to_csv(path, index=False)

In [76]:
path = join(data_path, 'Website results',
            'Quarterly index {}.csv'.format(export_quarter))
quarterly_index.to_csv(path, index=False)

In [77]:
path = join(data_path, 'Website results',
            'Annual index {}.csv'.format(export_quarter))
annual_index.to_csv(path, index=False)

### CO₂ intensity of each fuel category, plus files for website

In [113]:
gen_fuels_state.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),fuel category
0,COW,2001,1,3,8275496.0,852306.0,8275496.0,31.006900,-88.010300,1,772103776.8,772103776.8,772103776.8,772103776.8,Coal
1,COW,2001,1,7,587162.0,45991.0,587162.0,34.012800,-85.970800,1,54782214.6,54782214.6,54782214.6,54782214.6,Coal
2,COW,2001,1,8,6910786.0,706857.0,6910786.0,33.644344,-87.196486,1,644776333.8,644776333.8,644776333.8,644776333.8,Coal
3,COW,2001,1,10,3108892.0,320454.0,3108892.0,32.601700,-87.781100,1,290059623.6,290059623.6,290059623.6,290059623.6,Coal
4,COW,2001,1,26,9745127.0,1005460.0,9745127.0,33.244211,-86.458056,1,909220349.1,909220349.1,909220349.1,909220349.1,Coal


In [132]:
national_gen.tail()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
WWW  2017 8                     0.0        3879727.60                0.0
          9                     0.0        3357157.33                0.0
          10                    0.0        3558432.79                0.0
          11                    0.0        3512302.55                0.0
          12                    0.0        3831624.38                0.0

In [124]:
category_ef = reduce_emission_factors(ef)
category_ef

{'COW': 95.25, 'NG': 53.07, 'OOG': 59.0, 'PC': 102.1, 'PEL': 75.975}

In [133]:
extra_co2.head()

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
COW  2001 1           1.611286e+09        1.253069e+09
          2           1.425925e+09        1.102661e+09
          3           1.745032e+09        1.406286e+09
          4           1.288744e+09        9.881243e+08
          5           1.335026e+09        1.040365e+09

In [141]:
national_gen['elec fuel fossil co2 (kg)'] = 0
for fuel in category_ef.keys():
    national_gen.loc[idx[fuel, :, :], 'elec fuel fossil co2 (kg)'] = (national_gen.loc[idx[fuel, :, :], 'elec fuel (mmbtu)'] 
                                                              * category_ef[fuel])

national_gen['elec fuel fossil co2 (kg)'] = (national_gen['elec fuel fossil co2 (kg)'] 
                                             + extra_co2['elec fuel co2 (kg)'])

In [143]:
national_gen.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)  \
type year month                                                            
COW  2001 1            1.865791e+09       177287111.0       1.823460e+09   
          2            1.583642e+09       149735483.0       1.547593e+09   
          3            1.644567e+09       155269010.0       1.605915e+09   
          4            1.485098e+09       140670652.0       1.449735e+09   
          5            1.609283e+09       151592915.0       1.574620e+09   

                 elec fuel co2 (kg)  elec fuel fossil co2 (kg)  
type year month                                                 
COW  2001 1            1.736846e+11               1.749376e+11  
          2            1.474082e+11               1.485109e+11  
          3            1.529634e+11               1.543697e+11  
          4            1.380873e+11               1.390754e+11  
          5            1.499826e+11               1.510229e+11

In [138]:
gen_monthly.head()

,fuel category,year,month,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),adjusted co2 (kg),adjusted index (g/kwh),adjusted index (lb/mwh),datetime
0,Coal,2001,1,1.848875e+09,1.762754e+08,1.810304e+09,1,1.755757e+11,1.719629e+11,1.755757e+11,1.719629e+11,1.762533e+11,999.874627,2204.323602,2001-01-01
1,Coal,2001,2,1.568672e+09,1.488241e+08,1.536017e+09,1,1.490065e+11,1.459476e+11,1.490065e+11,1.459476e+11,1.501817e+11,1009.121784,2224.709885,2001-02-01
2,Coal,2001,3,1.626246e+09,1.541157e+08,1.591151e+09,1,1.544286e+11,1.511424e+11,1.544286e+11,1.511424e+11,1.547682e+11,1004.234117,2213.934533,2001-03-01
3,Coal,2001,4,1.471568e+09,1.398943e+08,1.439361e+09,2,1.397181e+11,1.367018e+11,1.397181e+11,1.367018e+11,1.399598e+11,1000.468383,2205.632597,2001-04-01
4,Coal,2001,5,1.595267e+09,1.507916e+08,1.563698e+09,2,1.515424e+11,1.485856e+11,1.515424e+11,1.485856e+11,1.519525e+11,1007.698474,2221.572055,2001-05-01


In [60]:
gen_monthly = (group_fuel_cats(gen_fuels_state, custom_fuel_cat,
                              'type', 'fuel category')
                               .groupby(['fuel category', 'year', 'month'])
                               .sum()
                               .reset_index())

generation_index(gen_monthly, national_index, ['year', 'month'])
add_quarter(gen_monthly)
gen_monthly.drop(columns=['plant id', 'lat', 'lon'], inplace=True)
gen_monthly.head()

,fuel category,year,month,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),adjusted co2 (kg),adjusted index (g/kwh),adjusted index (lb/mwh),datetime
0,Coal,2001,1,1.848875e+09,1.762754e+08,1.810304e+09,1,1.755757e+11,1.719629e+11,1.755757e+11,1.719629e+11,1.762533e+11,999.874627,2204.323602,2001-01-01
1,Coal,2001,2,1.568672e+09,1.488241e+08,1.536017e+09,1,1.490065e+11,1.459476e+11,1.490065e+11,1.459476e+11,1.501817e+11,1009.121784,2224.709885,2001-02-01
2,Coal,2001,3,1.626246e+09,1.541157e+08,1.591151e+09,1,1.544286e+11,1.511424e+11,1.544286e+11,1.511424e+11,1.547682e+11,1004.234117,2213.934533,2001-03-01
3,Coal,2001,4,1.471568e+09,1.398943e+08,1.439361e+09,2,1.397181e+11,1.367018e+11,1.397181e+11,1.367018e+11,1.399598e+11,1000.468383,2205.632597,2001-04-01
4,Coal,2001,5,1.595267e+09,1.507916e+08,1.563698e+09,2,1.515424e+11,1.485856e+11,1.515424e+11,1.485856e+11,1.519525e+11,1007.698474,2221.572055,2001-05-01


In [145]:
gen_monthly = (group_fuel_cats(national_gen.reset_index(), custom_fuel_cat,
                              'type', 'fuel category')
                               .groupby(['fuel category', 'year', 'month'])
                               .sum()
                               .reset_index())

generation_index(gen_monthly, national_index, ['year', 'month'])
add_quarter(gen_monthly)
# gen_monthly.drop(columns=['plant id', 'lat', 'lon'], inplace=True)
gen_monthly.head()

,fuel category,year,month,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),elec fuel co2 (kg),elec fuel fossil co2 (kg),adjusted co2 (kg),adjusted index (g/kwh),adjusted index (lb/mwh),datetime,quarter
0,Coal,2001,1,1.865791e+09,177287111.0,1.823460e+09,1.736846e+11,1.749376e+11,1.776697e+11,1002.157827,2209.357145,2001-01-01,1
1,Coal,2001,2,1.583642e+09,149735483.0,1.547593e+09,1.474082e+11,1.485109e+11,1.514844e+11,1011.679843,2230.349382,2001-02-01,1
2,Coal,2001,3,1.644567e+09,155269010.0,1.605915e+09,1.529634e+11,1.543697e+11,1.562857e+11,1006.547807,2219.035294,2001-03-01,1
3,Coal,2001,4,1.485098e+09,140670652.0,1.449735e+09,1.380873e+11,1.390754e+11,1.414209e+11,1005.333147,2216.357455,2001-04-01,2
4,Coal,2001,5,1.609283e+09,151592915.0,1.574620e+09,1.499826e+11,1.510229e+11,1.534224e+11,1012.068577,2231.206385,2001-05-01,2


In [146]:
gen_quarter = gen_monthly.groupby(['fuel category', 'year', 'quarter']).sum()
gen_quarter['adjusted index (g/kwh)'] = (gen_quarter['adjusted co2 (kg)']
                                         / gen_quarter['generation (mwh)'])
gen_quarter['adjusted index (lb/mwh)'] = gen_quarter['adjusted index (g/kwh)'] * 2.2046
gen_quarter['year_quarter'] = (gen_quarter.index.get_level_values('year').astype(str)
                               + ' Q'
                               + gen_quarter.index.get_level_values('quarter').astype(str))

gen_quarter.drop(columns=['month'], inplace=True)

gen_quarter.head()

total fuel (mmbtu)  generation (mwh)  \
fuel category year quarter                                         
Coal          2001 1              5.094000e+09       482291604.0   
                   2              4.822767e+09       454879371.0   
                   3              5.521643e+09       516334801.0   
                   4              4.793881e+09       450450163.0   
              2002 1              4.826986e+09       458892491.0   

                            elec fuel (mmbtu)  elec fuel co2 (kg)  \
fuel category year quarter                                          
Coal          2001 1             4.976968e+09        4.740562e+11   
                   2             4.716978e+09        4.492922e+11   
                   3             5.407625e+09        5.150763e+11   
                   4             4.687877e+09        4.465203e+11   
              2002 1             4.717666e+09        4.493577e+11   

                            elec fuel fossil co2 (kg)  adjusted co2 (kg)  \
fuel category year quarter                                                 
Coal          2001 1                     4.778182e+11       4.854397e+11   
                   2                     4.522444e+11       4.596810e+11   
                   3                     5.181080e+11       5.246841e+11   
                   4                     4.490041e+11       4.523317e+11   
              2002 1                     4.506617e+11       4.600400e+11   

                            adjusted index (g/kwh)  adjusted index (lb/mwh)  \
fuel category year quarter                                                    
Coal          2001 1                   1006.527406              2218.990320   
                   2                   1010.555733              2227.871169   
                   3                   1016.170400              2240.249265   
                   4                   1004.177080              2213.808791   
              2002 1                   1002.500601              2210.112824   

                           year_quarter  
fuel category year quarter               
Coal          2001 1            2001 Q1  
                   2            2001 Q2  
                   3            2001 Q3  
                   4            2001 Q4  
              2002 1            2002 Q1

In [147]:
gen_quarter.loc[idx[:, [2016, 2017], :], :].pivot_table(values='generation (mwh)', index='year_quarter', columns='fuel category')

fuel category,Coal,Hydro,Natural Gas,Nuclear,Other,Other Renewables,Solar,Wind
year_quarter,,,,,,,,
2016 Q1,2.783367e+08,77143435.03,3.124807e+08,204311810.0,12014944.08,19599002.47,9972423.08,60544038.75
2016 Q2,2.698418e+08,74601277.68,3.406956e+08,196483662.0,11217089.07,18871160.64,15221929.60,55950643.49
2016 Q3,3.860889e+08,57392554.18,4.319093e+08,207323928.0,10983016.57,20193989.75,17163262.54,47611323.49
2016 Q4,3.048812e+08,58674886.41,2.931986e+08,197574548.0,9864296.10,19869341.28,12306929.89,62886556.72
2017 Q1,2.917082e+08,82446476.08,2.671844e+08,201774183.0,10486886.26,19901616.03,13627415.87,68963728.97
2017 Q2,2.820069e+08,91780415.21,2.988013e+08,185066887.0,10189620.94,19341343.45,23871870.06,68090275.09
2017 Q3,3.459834e+08,65841424.07,4.004177e+08,211796355.0,10195268.75,20375956.56,23312119.15,46041120.63
2017 Q4,2.876587e+08,59505103.34,3.045420e+08,206312210.0,10483252.65,20049532.47,15954262.49,70888373.89


In [148]:
gen_annual = gen_monthly.groupby(['fuel category', 'year']).sum()
gen_annual['adjusted index (g/kwh)'] = (gen_annual['adjusted co2 (kg)']
                                         / gen_annual['generation (mwh)'])
gen_annual['adjusted index (lb/mwh)'] = gen_annual['adjusted index (g/kwh)'] * 2.2046

gen_annual.drop(columns=['month'], inplace=True)

gen_annual.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)  \
fuel category year                                                            
Coal          2001        2.023229e+10      1.903956e+09       1.978945e+10   
              2002        2.041797e+10      1.933130e+09       1.999690e+10   
              2003        2.078358e+10      1.973737e+09       2.036688e+10   
              2004        2.094025e+10      1.978301e+09       2.037575e+10   
              2005        2.135038e+10      2.012873e+09       2.080172e+10   

                    elec fuel co2 (kg)  elec fuel fossil co2 (kg)  \
fuel category year                                                  
Coal          2001        1.884945e+12               1.897175e+12   
              2002        1.904705e+12               1.911224e+12   
              2003        1.939946e+12               1.946207e+12   
              2004        1.940790e+12               1.945684e+12   
              2005        1.981363e+12               1.982397e+12   

                    adjusted co2 (kg)  adjusted index (g/kwh)  \
fuel category year                                              
Coal          2001       1.922137e+12             1009.548858   
              2002       1.951006e+12             1009.247234   
              2003       1.986245e+12             1006.337463   
              2004       1.968793e+12              995.193906   
              2005       1.994736e+12              990.989297   

                    adjusted index (lb/mwh)  quarter  
fuel category year                                    
Coal          2001              2225.651413       30  
              2002              2224.986452       30  
              2003              2218.571571       30  
              2004              2194.004484       30  
              2005              2184.735004       30

In [149]:
path = join(data_path, 'Website results',
            'Monthly generation {}.csv'.format(export_quarter))
gen_monthly.to_csv(path, index=False)

In [150]:
path = join(data_path, 'Website results',
            'Quarterly generation {}.csv'.format(export_quarter))
gen_quarter.to_csv(path)

In [151]:
path = join(data_path, 'Website results',
            'Annual generation {}.csv'.format(export_quarter))
gen_annual.to_csv(path)

In [152]:
path = join(data_path, 'Website results',
            '{} US Generation By Fuel Type.xlsx'.format(export_quarter))
writer = pd.ExcelWriter(path)

gen_monthly.to_excel(writer, sheet_name='Monthly', index=False)

gen_quarter.to_excel(writer, sheet_name='Quarterly')

gen_annual.to_excel(writer, sheet_name='Annual')
writer.save()

### Percent generation by custom fuel type

In [ ]:
df_list = []
for fuel in national_gen_cat.index.get_level_values('fuel category').unique():
    percent_gen = national_gen_cat.loc[fuel].divide(total_gen, fill_value=0)
    percent_gen['fuel category'] = fuel
    percent_gen.set_index('fuel category', inplace=True, append=True)
    df_list.append(percent_gen)
percent_gen = pd.concat(df_list)

In [ ]:
path = join(data_path, 'National data',
            'National generation {}.csv'.format(file_date))
national_gen_cat.to_csv(path)

path = join(data_path, 'National data',
            'National percent gen {}.csv'.format(file_date))
percent_gen.to_csv(path)

## Facility state and lat/lon file generation
~~Create a .csv with lat/lon and state code for each facility. Need to manually add the NERC region label.~~ ~~I've done this with a spatial join between the lat/lon and NERC shapefiles in QGIS. It should be possible to do the spatial join in [GeoPandas](http://geopandas.org/).  Could also add custom region labels (e.g. eGRID subregions, ISO/RTO boundaries, etc).~~

This isn't possible with a spatial join because NERC regions aren't strictly based on geography. Instead, I've taken the NERC codes from EIA-860 and assigned unknown plants (mostly those that retired before 2012 when modern NERCs were mostly defined, and those that were assigned a plant id in 2017) NERC labels using a k-nearest neighbors algorithm and lat/lon information.

See the `Assign NERC region labels` notebook for the creation of this file.

## Fraction of estimated gen/fuels in each NERC region

**NOTE** EIA changed the reporting requirements for wind and solar facilities in 2017. I'm now going to use the list of annual facilities from 2017 rather than from 2015, but still use the reported generation by those facilities in 2015.

In [ ]:
%aimport Data.make_data
from Data.make_data import get_annual_plants

In [ ]:
annual_ids_2015 = get_annual_plants(2015)

In [ ]:
annual_ids_2017 = get_annual_plants(2017)

Combine the lists of annual plants in 2015 and 2017. This lets us catch facilities that have gone from monthly to annual since 2015, but it also includes plants that were annual in 2015 an may have retired.

There is the possibility of an error in allocation to NERC regions for 2016 state-level generation when using the facilities that changed to annual in 2017. But since the state-level generation is so much smaller in 2016 I don't think this is much of an issue.

In [ ]:
annual_ids = set(annual_ids_2015.tolist() + annual_ids_2017.tolist())

In [ ]:
len(annual_ids)

### Add NERC region labels

In [ ]:
%aimport Analysis.state2nerc
from Analysis.state2nerc import fraction_state2nerc, add_region

In [ ]:
cwd = os.getcwd()
path = join(data_path, 'Facility labels',
            'Facility locations_knn.csv')
location_labels = pd.read_csv(path)

In [ ]:
nerc_state_path = join(data_path, 'Derived data',
                  'NERC_states.json')

with open(nerc_state_path, 'r') as f:
    nerc_states = json.load(f)

Added the filter that year must be 2015 - was getting all 2015 annual plants, but for all years!

In [ ]:
eia_2015_annual = eia_fac.loc[(eia_fac['plant id'].isin(annual_ids)) & 
                              (eia_fac['year'] == 2015)].copy()

# Group to state-level fuel categories
eia_2015_annual = group_fuel_cats(eia_2015_annual, state_fuel_cat)

In [ ]:
eia_2015_annual_nerc = add_facility_location(eia_2015_annual, location_labels, 
                                        labels=['state', 'nerc'])

This is 2015 data on annual reporting facilities (from both 2015 and 2017)

In [ ]:
# Get a list of all state abbreviations

all_states = []
for value in nerc_states.values():
    all_states.extend(value)
all_states = set(all_states)

In [ ]:
df_list = []

for state in all_states:
    try:
        df_list.append(fraction_state2nerc(eia_2015_annual_nerc,
                       state, region_col='nerc', fuel_col='type'))
    except:
        print(state)
        pass

In [ ]:
nerc_fraction = pd.concat(df_list)
nerc_fraction.set_index(['state', 'nerc', 'type'], inplace=True)
nerc_fraction.sort_index(inplace=True)

With the values below I can allocate extra state-level generation and fuel use to each of the NERC regions!

Making sure that no values are greater than 1 (within tolerance)

In [ ]:
(nerc_fraction.groupby(['state', 'type']).sum() > 1.0001).any()

## Allocate extra gen from the state-level to regions

I still need to generate state-level total generation and fuel use!

In [ ]:
idx = pd.IndexSlice

In [ ]:
# a dictionary to match column names
nerc_frac_match = {'% generation': 'generation (mwh)',
                   '% total fuel': 'total fuel (mmbtu)',
                   '% elec fuel': 'elec fuel (mmbtu)'}

### Load state-level total gen/fuel consumption

In [ ]:
path = join(data_path, 'Derived data',
            'EIA state-level gen fuel CO2 {}.csv'.format(file_date))

state_total = pd.read_csv(path, parse_dates=['datetime'])

In [ ]:
rename_cols(state_total)
state_total['state'] = state_total['geography'].str[-2:]

Simplify the dataframe

In [ ]:
cols = list(nerc_frac_match.values())
state_total = state_total.groupby(['state', 'year', 'month', 'type'])[cols].sum()

In [ ]:
# list of NERC regions
nercs = nerc_fraction.index.get_level_values('nerc').unique()

### Group the facility data to state fuel categories and add state labels

In [ ]:
cols = list(nerc_frac_match.values())
eia_fac_type = group_fuel_cats(eia_fac, state_fuel_cat)
eia_fac_type = add_facility_location(eia_fac_type, location_labels, ['state'])
eia_fac_type = eia_fac_type.groupby(['state', 'year', 'month', 'type'])[cols].sum()

### Calculate the extra gen/fuel consumption at the state levels
**Only worrying about extra generation from 2016 forward.**

In [ ]:
state_extra = (state_total.loc[idx[:, 2016:, :, :], :]
               - eia_fac_type.loc[idx[:, 2016:, :, :], :])
state_extra.dropna(how='all', inplace=True)
state_extra = state_extra.reorder_levels(['year', 'state', 'month', 'type'])
state_extra.sort_index(inplace=True)

Sort the index of each dataframe to make sure they can be easily combined.

In [ ]:
nerc_fraction.sort_index(inplace=True)

In [ ]:
state_extra.sort_index(inplace=True)

Create a copy of the `nerc_fraction` dataframe with repeated values for every month of the year, so that they MultiIndex matches the `state_extra` MultiIndex

In [ ]:
df_list = []
for month in range(1, 13):
    df = nerc_fraction.copy()
    df['month'] = month
    df.set_index('month', append=True, inplace=True)
    df_list.append(df)

nerc_frac_monthly = pd.concat(df_list, axis=0)
nerc_frac_monthly.sort_index(inplace=True)
nerc_frac_monthly = (nerc_frac_monthly
                     .reorder_levels(['nerc', 'state', 'month', 'type']))

Cycle through each year (2016 and 2017 in this case) and each NERC, multiplying the state-level extra generation, total fuel consumption, and fuel consumption for electricity by the share that should be allocated to each NERC.

In [ ]:
df_list_outer = []
for year in [2016, 2017]:
    df_list_inner = []
    for nerc in nercs:
        df = pd.concat([(nerc_frac_monthly
                         .loc[nerc]['% generation']
                         * state_extra
                         .loc[year]['generation (mwh)']).dropna(),
                        (nerc_frac_monthly.
                         loc[nerc]['% total fuel']
                         * state_extra
                         .loc[year]['total fuel (mmbtu)']).dropna(),
                        (nerc_frac_monthly
                         .loc[nerc]['% elec fuel']
                         * state_extra
                         .loc[year]['elec fuel (mmbtu)']).dropna()],
                        axis=1)
        df.columns = nerc_frac_match.values()
        df['nerc'] = nerc
        df['year'] = year
        df = df.groupby(['year', 'nerc', 'month', 'type']).sum()
        df_list_inner.append(df)

    df_list_outer.append(pd.concat(df_list_inner))
final = pd.concat(df_list_outer)
final.sort_index(inplace=True)

In [ ]:
nerc_frac_monthly.sort_index(inplace=True)

Although the dataframe is called `final`, it's really just the final allocated extra state-level generation/fuel consumption

In [ ]:
path = join(data_path, 'Derived data',
            'NERC extra gen fuels {}.csv'.format(file_date))
final.to_csv(path)